In [ ]:
# !pip install bert_score

In [ ]:
import pandas as pd
from bert_score import score, BERTScorer
import matplotlib.pyplot as plt
from transformers import pipeline

In [ ]:
# df = pd.read_csv('../eval/distilgpt2_eval.csv')
# model_name = 'distilgpt2'
# df = pd.read_csv('../eval/distilgpt2-finetuned-textbook_dataset_eval.csv')
# model_name = 'distilgpt2-finetuned'
# df = pd.read_csv('../eval/falcon7b_rag_eval.csv')
# model_name = 'falcon7b_rag'
df = pd.read_csv('../eval/falcon7b_finetuned_eval.csv')
model_name = 'falcon7b_finetuned'

In [ ]:
df

In [ ]:
#questions = df['Question'].tolist()
ideal_answers = df['Expected Answer'].tolist()
gen_answers = df['Actual Answer'].tolist()

# questions = df['question'].tolist()
# ideal_answers = df['Expected'].tolist()
# gen_answers = df['Generated'].tolist()

ideal_answers = [str(x) for x in ideal_answers]
gen_answers = [str(x) for x in gen_answers]

In [ ]:

scorer = BERTScorer(lang="en", rescale_with_baseline=True)


P, R, F1 = scorer.score(gen_answers, ideal_answers)

# print(f"System level F1 score: {F1.mean():.3f}")


# plt.hist(F1, bins=20)
# plt.xlabel("Score")
# plt.ylabel("Counts")
# plt.title('Precision, Recall, and F1 Score for Each Data Point')
# plt.show()


In [ ]:
df['BERTScore_f1'] = F1
df['BERTScore_precision'] = P
df['BERTScore_recall'] = R

df.to_csv(f'../eval/{model_name}_eval.csv', index=False)

In [ ]:
print(f"System level F1 score: {F1.mean():.3f}")

plt.hist(F1, bins=20)
plt.xlabel("Score")
plt.ylabel("Counts")
plt.title(f'{model_name} F1 Score Distribution')
# plt.suptitle(f'Mean F1 Score: {F1.mean():.3f}')
plt.style.use('ggplot')
plt.savefig(f'../eval/{model_name}_f1_score_distribution.png')
plt.show()

In [ ]:
import numpy as np

P = np.array(P)
R = np.array(R)
F1 = np.array(F1)


num_points = len(P)


idx = np.arange(num_points)


bar_width = 0.2


plt.figure(figsize=(12, 6))


plt.bar(idx, P, bar_width, label='Precision')
plt.bar(idx + bar_width, R, bar_width, label='Recall')
plt.bar(idx + 2 * bar_width, F1, bar_width, label='F1 Score')

plt.xlabel('Data Points')
plt.ylabel('Scores')
plt.title('Precision, Recall, and F1 Score for Each Data Point')
plt.xticks(idx + bar_width, idx)


plt.legend()


plt.tight_layout()
plt.style.use('ggplot')
plt.show()


In [ ]:


average_precision = P.mean()
average_recall = R.mean()
average_f1 = F1.mean()

scores = [average_precision, average_recall, average_f1]
labels = ['Precision', 'Recall', 'F1 Score']

plt.figure(figsize=(8, 6))
barlist = plt.bar(labels, scores, color=['blue', 'green', 'red'])
for idx, bar in enumerate(barlist):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height, f'{scores[idx]:.3f}', ha='center', va='bottom')
plt.xlabel('Metrics')
plt.ylabel('Scores')
plt.title(f'{model_name} Average Precision, Recall, and F1 Score')
plt.style.use('ggplot')
plt.savefig(f'../eval/{model_name}_average_scores.png')
plt.show()
